# Assignment 3: EEG and Event-Related Potentials
Please submit this assignment to Canvas as a jupyter notebook (.ipynb).  The assignment will introduce you to EEG brain data, as well as some of the techniques and methods used to analyze it.

In [1]:
# imports
import pandas as pd
import cmlreaders as cml
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Working with EEG data
We learned how to load basic information about CML experiments and experimental events in assignment 1. Next, we're going to load EEG/iEEG data that correspond to those events.

## Question 1: Electrodes in the Brain
For this question, we will use the term "electrode" to refer to individual contacts, as opposed to bipolar referenced virtual "electrodes".  In other words, use the contacts dataframe (not the pairs dataframe) to answer the following.

1) How many electrodes does R1111M have placed in the temporal cortex?
2) How many electrodes does R1111M have on the left hemisphere and how many on the right hemisphere?

In [5]:
# Question 1.1
### YOUR CODE HERE

In [6]:
# Question 1.2
### YOUR CODE HERE

## Question 2
Now we will compare z-scored voltage traces for encoded words that were subsequently recalled vs. not recalled.  This sort of "Subsequent Memory Effect" (SME) analysis is of great interest, since it allows us to get at the question of what neural features underlie successful memory.

1) Plot the Z-scored ERP for a 2-second span (-400 ms to 1600 ms) surrounding each word encoding event for a lateral occipital electrode from R1383J's first FR1 session.  We want to two lines on our plot, one averaging over words that were subsequently recalled, and one averaging over words that were not later recalled.
* Add vertical and horizontal lines on your plot to indicate word onset and Z = 0, respectively.
* Add a legend and label each line (Rec v. NRec)


2) Does the occipital electrode show an effect?

In [17]:
# Question 2.1
### YOUR CODE HERE

Question 2.2

**YOUR ANSWER HERE**

## Event Related Potentials (ERPs)

For the remaining exercises in this assignment, we will be analyzing neural responses to events using Event Related Potentials (ERPs).  You will be working with data from a single electrode 75 (Oz) (labeled 'E75') in a single subject LTP093 for LTPFR2 experimental sessions. In this and all subsequent exercises, you will analyze the time series of data surrounding each word presentation, from 200 ms prior to word onset until 1,000 ms after word onset. To remove electrical signals that do not reflect neural activity, we use the Localized Component Filtering method (DelPozo-Banos & Weidemann, 2017). To load the LCF-cleaned
data you will pass the clean=True argument to the CMLReader’s load_eeg() method.

A few points:

* The LTPFR2 data set you'll be working with for this assignment is a *scalp EEG* data set and has slightly different column definitions than intracranial EEG sessions.  For example, the 'trial' column indicates the list number of the event.
* If you use array indexing to access electrode 75 (as opposed to e.g., xarray indexing), make sure to double check the index for this contact.
* Load the scalp EEG data with the `load_eeg()` function by setting the argument `clean=True` and simply don't include the `scheme` argument in the function call to `load_eeg()`. The scheme is only needed for intracranial data and indicates whether monopolar or bipolar-rereferenced EEG should be loaded and for which contacts/bipolar pairs.
* Make sure to subset the events to just the WORD or REC_WORD events before loading the EEG. CMLReaders won't load overlapping EEG intervals (i.e. epochs) surrounding events, which will happen if you try to load EEG surrounding all the behavioral events.  Here is some example code:

In [18]:
exp = 'ltpFR2'                      # LTPFR2 experiment
df = cml.get_data_index('ltp')
df = df.query("experiment == @exp")     # select only LTPFR2 sessions
df

,all_events,experiment,import_type,math_events,original_session,session,subject,subject_alias,task_events
487,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,0,0,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
488,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,1,1,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
489,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,10,10,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
490,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,11,11,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
491,protocols/ltp/subjects/LTP093/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP093/experiments/ltpF...,12,12,LTP093,LTP093,protocols/ltp/subjects/LTP093/experiments/ltpF...
...,...,...,...,...,...,...,...,...,...
6526,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,5,5,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
6527,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,6,6,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
6528,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,7,7,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...
6529,protocols/ltp/subjects/LTP393/experiments/ltpF...,ltpFR2,build,protocols/ltp/subjects/LTP393/experiments/ltpF...,8,8,LTP393,LTP393,protocols/ltp/subjects/LTP393/experiments/ltpF...


In [19]:
sess_df = df.iloc[0]  # grab meta-data for first session in LTPFR2 data set

# get a data reader as before, but without the montage and localization arguments
reader = cml.CMLReader(subject=sess_df.subject,
                       session=sess_df.session,
                       experiment=sess_df.experiment)

# load the behavioral events for this session
evs = reader.load('events')
word_evs = evs[evs['type'] == 'WORD']

# load the eeg --> for scalp EEG we want to specify clean = True)
eeg = reader.load_eeg(events=word_evs, rel_start = -100, rel_stop = 1000, clean = True).to_ptsa()
eeg

Opening raw data file /protocols/ltp/subjects/LTP093/experiments/ltpFR2/sessions/0/ephys/current_processed/LTP093 20140902 0959.2_clean_raw.fif...
    Range : 0 ... 2858664 =      0.000 ...  5717.328 secs
Ready.
Reading 0 ... 2858664  =      0.000 ...  5717.328 secs...
Not setting metadata
576 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 576 events and 551 original time points ...
0 bad epochs dropped


<xarray.TimeSeries (event: 576, channel: 124, time: 551)>
array([[[-5.44548224e-06, -2.31033754e-06, -1.26133752e-06, ...,
         -5.77436827e-06, -6.94353150e-06, -8.80198476e-06],
        [-9.38845537e-06, -7.86169575e-06, -4.78514575e-06, ...,
          1.52607917e-06, -3.10387853e-08, -9.07887552e-06],
        [ 1.38913650e-05,  1.45578575e-05,  7.86829994e-06, ...,
          1.09732179e-05,  1.07329454e-05, -3.39196180e-06],
        ...,
        [ 4.16697930e-06,  5.21644972e-06,  3.70095133e-06, ...,
          5.76517640e-07, -1.17582011e-06, -5.81043910e-06],
        [-2.05874347e-06,  2.71453112e-07, -1.06465220e-06, ...,
         -6.86545704e-06, -7.56921814e-06, -6.37896155e-06],
        [-7.55552826e-07, -8.64813683e-07,  2.31394246e-07, ...,
         -4.81928872e-06, -8.23925779e-06, -8.04741857e-06]],

       [[ 1.17181320e-05,  1.56529312e-05,  1.47506504e-05, ...,
          7.07020424e-06,  3.26276445e-06, -3.31738638e-06],
        [ 1.91926217e-06,  1.01686153e-05,  5.45040988e-06, ...,
          4.89050435e-06,  4.83034550e-07,  2.91767477e-06],
        [-1.62102127e-06,  4.43498896e-06, -3.58390426e-06, ...,
          6.82189463e-06,  6.18200968e-06,  6.07595681e-06],
...
        [ 1.68264544e-06,  1.70072853e-06, -8.53746412e-07, ...,
         -6.44807099e-06, -8.84175108e-06, -6.16033123e-06],
        [ 2.35205864e-06, -4.95025109e-06, -2.16679739e-06, ...,
         -4.91195916e-06, -3.88908624e-06, -3.86504005e-06],
        [-4.24452214e-07,  6.88936783e-08, -8.46136627e-07, ...,
         -5.32955645e-06, -7.12135122e-06, -6.36636303e-06]],

       [[-3.99960064e-06, -1.03470564e-05, -1.15901107e-05, ...,
         -1.62907867e-05, -1.49298906e-05, -1.71170005e-05],
        [-1.00183573e-05, -7.59011553e-06, -3.26234030e-06, ...,
         -1.77438373e-05, -1.38395786e-05, -1.09123583e-05],
        [-9.97141359e-06, -5.90627049e-06, -6.82755409e-07, ...,
         -1.02412081e-05, -7.93762157e-06, -9.34263608e-06],
        ...,
        [-4.08690118e-06, -2.65405154e-06, -1.78929519e-06, ...,
         -1.13761902e-05, -1.02451219e-05, -1.07157964e-05],
        [-3.82428359e-06, -3.91137337e-06, -3.48279213e-06, ...,
         -1.44409961e-05, -1.35250644e-05, -1.34141922e-05],
        [-1.84810137e-06, -3.84987264e-07,  8.30413876e-07, ...,
         -1.12372148e-06, -3.50123524e-06, -3.74488067e-06]]])
Coordinates: (12/32)
  * event               (event) object MultiIndex
  * answer              (event) int64 -999 -999 -999 -999 ... -999 -999 -999
  * begin_distractor    (event) int64 0 0 0 0 0 ... 24000 24000 24000 24000
  * begin_math_correct  (event) int64 -999 -999 -999 -999 -999 ... 12 12 12 12
  * eegfile             (event) object '/protocols/ltp/subjects/LTP093/experi...
  * eegoffset           (event) int64 285929 287177 288442 ... 2768352 2769567
    ...                  ...
  * test                (event) object (0, 0, 0) (0, 0, 0) ... (0, 0, 0)
  * trial               (event) int64 1 1 1 1 1 1 1 1 ... 24 24 24 24 24 24 24
  * type                (event) object 'WORD' 'WORD' 'WORD' ... 'WORD' 'WORD'
  * channel             (channel) <U4 'E1' 'E2' 'E3' ... 'E124' 'E125' 'E128'
  * time                (time) float64 -100.0 -98.0 -96.0 ... 996.0 998.0 1e+03
    samplerate          float64 500.0

## Question 3

1) For each word encoding event in the first list of the first session of the same subject (LTP093) from the previous problem, plot the individual voltage timeseries from 250 ms before the onset of the word until 1000 ms after the onset of the study word. In each figure panel mark study word onset. 

2) Can you identify any consistent patterns in the voltage time series between serial positions just by looking at them?

In [20]:
# Question 3.1
### YOUR CODE HERE

Question 3.2

**YOUR CODE HERE**

## Question 4
1) Generate ERPs for all the events in the first session (subject LTP093) for the first, second, and third serial positions separately (one ERP per serial position) and then an ERP averaged across the events in the remaining serial positions 4 - 24 in Session 1. To carry out an ERP analysis on the voltage data, we must 
    1. Load the desired events.
    2. Filter to only encoding events.
    3. Get the voltage for all encoding events.
    4. Baseline correct the voltage traces separately for each trace by establishing a "baseline" period of the signal from 250 ms before stimulus onset (word presentation) to 50ms before the stimulus onset (i.e., the "baseline" voltage), and then subtracting the average of the baseline from all values in the trace, and dividing all of these resulting values by the standard deviation of the baseline period.
        * Baseline correction accounts for differing electrical characteristics of each channel, and for the fact that the electrical activity (or roughly speaking, state) of the brain will differ from event to event when the stimulus is presented. What we care about is how much the stimulus impacts the electrical activity of the brain relative to what it was just prior to the stimulus.
    5. Filter events by serial position.
    6. Plot the average voltage traces for the different groups of serial positions together on a single figure. 
2) Compare the ERPs to the individual time series you plotted earlier. Is it easier to distinguish the serial positions of items when including data from all events?

In [21]:
# Question 4.1
### YOUR CODE HERE

Question 4.2

**YOUR ANSWER HERE**

## Question 5
1) Again generate ERPs comparing items across the groups of serial positions given in Problem 4, but this time averaging over data from all sessions from the same subject. 

2) Compare the ERPs obtained here with those obtained in the previous exercise.

In [22]:
# Question 5.1
### YOUR CODE HERE

Question 5.2

**YOUR ANSWER HERE**